# Tweepy is a twitter api wrapper in python, it has an extensive documentation and hence can be understood easiely. #


# consumer_key, consumer_secret, access_token and access_secret can be achieved by making a twitter app.

In [ ]:
import tweepy
consumer_key = 'Insert Here'
consumer_secret = 'Insert Here'
access_token = 'Insert Here'
access_secret = 'Insert Here'

In [ ]:
#the keys collected above are used for authentication purposes
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [ ]:
api = tweepy.API(auth)

In [ ]:
#pandas was used to make it easier to read the excel file.
import pandas as pd
df = pd.read_excel("Twitter_handles.xlsx")
print(df.columns)
handles = df['Twitter Handles (ICO)'].values

In [ ]:
#gives you the number of requests remaining and the refresh time limit.
api.rate_limit_status()

In [ ]:
'''
Used to handle the RateLimitError in twitter api,
read about this in tweepy doc.
'''

import time
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.TweepError:
            print("Let's wait for some time")
            #15 minutes is the refresh time for fetching follower ids
            time.sleep(15 * 60)

In [ ]:
#sqlite3 is a SQLite implementation for python. You can use any other library of your choice.
import sqlite3 as sql
#this will create a database if not already present
conn = sql.connect('twitter_handles.db')
c = conn.cursor()

In [ ]:
#make a table inside the database to store the values.
c.execute('''CREATE TABLE handles
             (ID INTEGER PRIMARY KEY, 
              Follower_Source TEXT NOT NULL, 
              Follower_IDs TEXT UNIQUE);''')

In [ ]:
# tweepy cursors are used for better handling of RateLimitError
for handle in handles:
    print("Current Handle: " + handle)
    with open("handles_2.txt", mode='w') as file:
        for follower in limit_handled(tweepy.Cursor(api.followers_ids, id=handle).items()):
            file.write(str(follower))
            file.write("\n")
            
    print("Followers of " + handle + " collected, Lets start inserting..\n")
    #Insert all the followers collected into a text file        
    lst = []
    follower_source = handle
    with open("handles_2.txt", mode='r') as file:
        for f in file:
            f = f[:-1]
            lst.append((follower_source, f))
                    
    c.executemany("INSERT or IGNORE INTO handles(Follower_Source, Follower_IDs) VALUES (?,?)", lst)
    conn.commit()
        
                
        
        

In [ ]:
# necessary for saving the changes to the database.
conn.commit()

In [ ]:
#To display the database by copying it in a .csv file.
import csv
with open("user_obj.csv", mode='w', newline='') as csvfile:
    write = csv.writer(csvfile)
    for row in c.execute("SELECT * FROM handles"):
        write.writerow(row)
    
    

In [ ]:
conn.commit()